In [1]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

tokenizer = BertTokenizer.from_pretrained('models')

In [2]:
import pandas as pd

df_micro = pd.read_csv('diachrony_for_russian/datasets/micro.csv')

In [3]:
def get_sent(WORD, year):
    word_sent = []
    with open('data/proc_data_{}.txt'.format(str(year)), 'r') as f:
        for l in f.readlines():
            try:
                if WORD in l.split():
                    word_sent.append(l)
            except:
                print(w, l)
          
    return word_sent

In [4]:
def get_emb(WORD, word_sent, model):
    emb = []
    for s in word_sent:
        try:
            idx = s.split().index(WORD)
            input_ids = tf.constant(tokenizer.encode(s.split()))[None, :]
            outputs = model(input_ids)[0]
            embed = outputs[:, idx + 1, :] 
            emb.append(embed.numpy().reshape(-1))
        except:   
            pass
    return emb

In [6]:
K = 5

In [7]:
data_fix = {}

In [ ]:
from scipy.spatial import distance
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np

for y in range(2000, 2015):
    model1 = model2
    model2 = TFBertModel.from_pretrained('models/model_' + str(y + 1), from_pt=True)
    words = df_micro.loc[df_micro['BASE_YEAR'] == y]
    print(y)
    for idx, row in words.iterrows():
        WORD = row['WORD'][:-4]
        year = row['BASE_YEAR']
        label = row['GROUND_TRUTH']
        if WORD + str(year) in data_fix:
            continue

        word_sent = get_sent(WORD, year)
        c1 = len(word_sent)
        word_sent = word_sent[:100]
        print('Find {} word usage'.format(len(word_sent)))
        emb = get_emb(WORD, word_sent, model1)
        print('Make {} word embeddings'.format(len(emb)))

        word_sent1 = get_sent(WORD, year + 1)
        c2 = len(word_sent1)
        word_sent1 = word_sent1[:100]
        print('Find {} word usage'.format(len(word_sent1)))
        emb1 = get_emb(WORD, word_sent1, model2)
        print('Make {} word embeddings'.format(len(emb1)))
        all_p = []
        all_p.extend(emb)
        all_p.extend(emb1)
        if len(emb) == 0 or len(emb1) == 0:
            continue

        kmeans = KMeans(n_clusters=K, random_state=0).fit(all_p)
        labels = kmeans.labels_

        p1 = []
        for i in range(K):
            l = kmeans.labels_[:len(emb)]
            p1.append(np.sum(l == i) / len(l))
        p2 = []
        for i in range(K):
            l = kmeans.labels_[len(emb):]
            p2.append(np.sum(l == i) / len(l))
            
        data_fix[WORD + str(year)] = [p1, p2, c1, c2, label]
    
    del model1

In [70]:
X, y = [], []

In [ ]:
for idx, row in df_micro.iterrows():
    WORD = row['WORD'][:-4]
    year = row['BASE_YEAR']
    if WORD + str(year) in data_fix:
        p1 = data_fix[WORD + str(year)][0]
        p2 = data_fix[WORD + str(year)][1]
        c1 = data_fix[WORD + str(year)][2]
        c2 = data_fix[WORD + str(year)][3]
        l = data_fix[WORD + str(year)][4]
        X.append([np.max(np.square(np.array(p1) - np.array(p2))),
                  distance.jensenshannon(p1, p2), c1, c2
                 ])
        y.append(l)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score as f1
res_score = []
for i in range(9):
    n = 30
    x_train = X[:i * n] + X[(i + 1) * n:]
    y_train = y[:i * n] + y[(i + 1) * n:]
    x_test = X[i * n:(i + 1) * n]
    y_test = y[i * n:(i + 1) * n]
    
    clf = LogisticRegression(random_state=42, max_iter=10000, class_weight='balanced')
    clf.fit(np.array(x_train), y_train)
    pred = clf.predict(np.array(x_test))
    res_score.append(f1(pred, y_test, average='macro'))

print(np.mean(res_score))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score as f1
res_score = []
for i in range(9):
    n = 30
    x_train = X[:i * n] + X[(i + 1) * n:]
    y_train = y[:i * n] + y[(i + 1) * n:]
    x_test = X[i * n:(i + 1) * n]
    y_test = y[i * n:(i + 1) * n]
    
    clf = LogisticRegression(random_state=42, max_iter=10000)
    clf.fit(np.array(x_train), y_train)
    pred = clf.predict(np.array(x_test))
    res_score.append(f1(pred, y_test, average='macro'))

print(np.mean(res_score))